In [3]:
# Scrapping WOS v.1

from wos import WosClient
import wos.utils

with WosClient(SID='f312e2fe4d304f467ec3bd7d2af46ea75c5817d7') as client:
    print(wos.utils.query(client, 'AU=Knuth Donald'))

<?xml version="1.0" encoding="UTF-8"?>
<SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:ns0="http://schemas.xmlsoap.org/soap/envelope/" xmlns:ns1="http://woksearch.v3.wokmws.thomsonreuters.com">
   <SOAP-ENV:Header/>
   <ns0:Body>
      <ns1:search>
         <queryParameters>
            <databaseId>WOS</databaseId>
            <userQuery>AU=Knuth Donald</userQuery>
            <queryLanguage>en</queryLanguage>
         </queryParameters>
         <retrieveParameters>
            <firstRecord>1</firstRecord>
            <count>5</count>
            <sortField>
               <name>RS</name>
               <sort>D</sort>
            </sortField>
         </retrieveParameters>
      </ns1:search>
   </ns0:Body>
</SOAP-ENV:Envelope>
<?xml version="1.0" encoding="UTF-8"?>
<SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" 

WebFault: b"Server raised fault: 'Session not found: SID=f312e2fe4d304f467ec3bd7d2af46ea75c5817d7'"

In [4]:
# Scrapping WOS v.2
# Package outdated

import time
import clarivate.wos_starter.client
from pprint import pprint
from clarivate.wos_starter.client.apis.tags import documents_api
from clarivate.wos_starter.client.model.document import Document
from clarivate.wos_starter.client.model.documents_list import DocumentsList

ModuleNotFoundError: No module named 'clarivate'

In [ ]:
# Scrapping Sci-Hub

from scihub import SciHub

sh = SciHub()

# exactly the same thing as fetch except downloads the articles to disk
# if no path given, a unique name will be used as the file name
result = sh.download('https://onlinelibrary.wiley.com/doi/full/10.1111/gcb.12569', path='paper.pdf')